In [1]:
'''
1. データセットの読み込みと正規化
'''
# tensorflow.keras のインポート
from tensorflow import keras

# Fashion-MNISTデータセットの読み込み
(x_train, t_train), (x_test, t_test) = keras.datasets.fashion_mnist.load_data()

# 訓練データを正規化
x_train = x_train / 255
# テストデータを正規化
x_test = x_test / 255

In [2]:
'''
2.モデルの生成
'''
# モデルオブジェクトを生成
model = keras.Sequential([
    # 入力するテンソルの形状を(28, 28)から(784)にフラット化する
    keras.layers.Flatten(input_shape=(28, 28)),
    # 隠れ層のユニット数:256
    # 活性化関数:ReLU
    keras.layers.Dense(256, activation='relu'),
    # 出力層のニューロン数:10
    # 活性化関数:ソフトマックス 
    keras.layers.Dense(10, activation='softmax')
])

# 学習率
learning_rate = 0.1                  
# モデルのコンパイル
model.compile(
    # 損失関数はスパースラベル対応クロスエントロピー誤差
    loss='sparse_categorical_crossentropy',
    # オプティマイザーはAdam
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    # 学習評価として正解率を指定
    metrics=['accuracy']
    )

model.summary() # モデルのサマリー(概要)を出力

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               200960    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 203,530
Trainable params: 203,530
Non-trainable params: 0
_________________________________________________________________


In [3]:
%%time
'''
3. モデルの学習
'''
epoch = 100
# ミニバッチのサイズ
batch_size = 64

history = model.fit(
    x_train,               # 訓練データ
    t_train,               # 正解ラベル
    epochs=epoch,          # エポック数を設定
    batch_size=batch_size, # ミニバッチのサイズを設定
    verbose=1,             # 進捗状況を出力
    validation_split=0.2   # 20パーセントのデータを検証に使用
    )

Train on 48000 samples, validate on 12000 samples
Epoch 1/100
48000/48000 [==============================] - 3s 69us/sample - loss: 0.5225 - accuracy: 0.8196 - val_loss: 0.4493 - val_accuracy: 0.8384
Epoch 2/100
48000/48000 [==============================] - 3s 57us/sample - loss: 0.3866 - accuracy: 0.8615 - val_loss: 0.3729 - val_accuracy: 0.8646
Epoch 3/100
48000/48000 [==============================] - 3s 57us/sample - loss: 0.3489 - accuracy: 0.8723 - val_loss: 0.3472 - val_accuracy: 0.8768
Epoch 4/100
48000/48000 [==============================] - 3s 60us/sample - loss: 0.3213 - accuracy: 0.8838 - val_loss: 0.3457 - val_accuracy: 0.8753
Epoch 5/100
48000/48000 [==============================] - 3s 60us/sample - loss: 0.2985 - accuracy: 0.8908 - val_loss: 0.3340 - val_accuracy: 0.8803
Epoch 6/100
48000/48000 [==============================] - 3s 57us/sample - loss: 0.2828 - accuracy: 0.8955 - val_loss: 0.3280 - val_accuracy: 0.8842
Epoch 7/100
48000/48000 [=========================

Epoch 55/100
48000/48000 [==============================] - 3s 61us/sample - loss: 0.0662 - accuracy: 0.9753 - val_loss: 0.4892 - val_accuracy: 0.8946
Epoch 56/100
48000/48000 [==============================] - 3s 58us/sample - loss: 0.0669 - accuracy: 0.9751 - val_loss: 0.4839 - val_accuracy: 0.8978
Epoch 57/100
48000/48000 [==============================] - 3s 57us/sample - loss: 0.0634 - accuracy: 0.9757 - val_loss: 0.5160 - val_accuracy: 0.8932
Epoch 58/100
48000/48000 [==============================] - 3s 59us/sample - loss: 0.0621 - accuracy: 0.9771 - val_loss: 0.5262 - val_accuracy: 0.8899
Epoch 59/100
48000/48000 [==============================] - 3s 56us/sample - loss: 0.0624 - accuracy: 0.9770 - val_loss: 0.5218 - val_accuracy: 0.8939
Epoch 60/100
48000/48000 [==============================] - 3s 56us/sample - loss: 0.0579 - accuracy: 0.9783 - val_loss: 0.5282 - val_accuracy: 0.8947
Epoch 61/100
48000/48000 [==============================] - 3s 56us/sample - loss: 0.0578 - ac

In [4]:
# モデルとパラメーターの値を保存
# モデルをmodel.jasonとして保存
with open('model.json', 'w') as json_file:
    json_file.write(model.to_json())
# パラメーターをweight.h5として保存
model.save_weights('weight.h5')

In [7]:
from tensorflow.keras.models import model_from_json

# モデルの読み込み
model_r = model_from_json(open('model.json', 'r').read())

# 復元したモデルのコンパイル
model_r.compile(loss='sparse_categorical_crossentropy',
                optimizer=keras.optimizers.Adam(learning_rate=0.001),
                metrics=['accuracy']
                )
# 重みの読み込み
model_r.load_weights('weight.h5')

In [9]:
# テストデータで保存済みのモデルを評価
test_loss, test_acc = model_r.evaluate(x_test, t_test, verbose=1)

10000/10000 [==============================] - 0s 47us/sample - loss: 0.8120 - accuracy: 0.8853
